<h1>Полученное задание</h1>

**ДЗ-2. ЗАДАЧА 2.**

Решить классическую задачу линейного программирования: найти минимальное возможное значение функции f(x) на множестве X допустимых значений переменных, заданном неравенствами x1 >= 0, x2 >= 0, x3 >= 0, x4 >= 0, x5 >= 0, и уравнениями (У).

Для этого (1) найти первую угловую точку методом искусственного базиса,  
(2) с помощью модифицированного симплекс метода найти угловую точку, на которой достигается точная нижняя грань целевой функции. Выписать последовательные симплекс таблицы, угловые точки и их базисы. Записать координаты итоговой угловой точки и значение целевой функции в ней. 

**Вариант 21. Якимова Т. С.**

f = 3 x1 + x2 + 7 x3 + 6 x4 + 3 x5;

(У)   -9x1 - 3x2 + 20x3 + 23x4 - 7x5 = 2, 

 -9x1 + 15x2 - 9x3 + 23x4 + 10x5 = 8.


In [11]:
import numpy as np

In [12]:
def print_vector(name, vector):
    return f"{name} = [{', '.join(f'{x:.4f}' for x in vector)}]"

In [13]:
def print_basis(basis):
    return f"Basis: {basis}"

In [14]:
def print_simplex_table(A, B_inv, basis, xb, c, z):
    print("\nSimplex Table:")
    print(f"Basic Variables: {basis}")
    print(f"Basic Solution: {print_vector('xb', xb)}")
    print(f"Objective Value: z = {z:.4f}")
    print("Reduced Costs (for non-basic variables):")
    for j in range(A.shape[1]):
        if j not in basis:
            cj_bar = c[j] - c[basis].dot(B_inv.dot(A[:, j]))
            print(f"c{j+1}_bar = {cj_bar:.4f}")

In [15]:
def artificial_basis(A, b, m, n):
    print("\n=== Artificial Basis Method ===")
    A_art = np.hstack([A, np.eye(m)])
    c_art = np.zeros(n + m)
    c_art[n:n+m] = 1  # Objective: minimize sum of artificial variables
    basis = list(range(n, n + m))  # Initial basis: artificial variables
    B_inv = np.eye(m)
    xb = b.copy()
    
    print(f"Initial Basic Solution: {print_vector('x', np.zeros(n + m))}")
    print(f"Artificial Variables: x{n+1}, ..., x{n+m}")
    print(print_basis(basis))
    
    # Revised Simplex for Artificial Problem
    while True:
        cb = c_art[basis]
        z = cb.dot(xb)
        print_simplex_table(A_art, B_inv, basis, xb, c_art, z)
        
        # Compute reduced costs
        min_cj_bar = float('inf')
        entering = -1
        for j in range(n + m):
            if j not in basis:
                cj_bar = c_art[j] - cb.dot(B_inv.dot(A_art[:, j]))
                if cj_bar < min_cj_bar:
                    min_cj_bar = cj_bar
                    entering = j
        if min_cj_bar >= -1e-10:  # Optimality condition
            break
            
        # Compute direction
        y = B_inv.dot(A_art[:, entering])
        if all(y <= 1e-10):
            raise ValueError("Problem is infeasible")
        
        # Compute step length
        ratios = [xb[i] / y[i] if y[i] > 1e-10 else float('inf') for i in range(m)]
        theta = min(r for r in ratios if r > 0)
        leaving_idx = ratios.index(theta)
        leaving = basis[leaving_idx]
        
        # Update basis
        basis[leaving_idx] = entering
        E = np.eye(m)
        for i in range(m):
            if i != leaving_idx:
                E[i, leaving_idx] = -y[i] / y[leaving_idx]
            else:
                E[i, leaving_idx] = 1 / y[leaving_idx]
        B_inv = E.dot(B_inv)
        xb = B_inv.dot(b)
    
    # Check if artificial variables are zero
    if abs(c_art[basis].dot(xb)) > 1e-10:
        raise ValueError("No feasible solution exists")
    
    # Remove artificial variables
    basis = [j for j in basis if j < n]
    if len(basis) < m:
        raise ValueError("Degeneracy or infeasibility detected")
    
    print("\nFound Initial Basic Feasible Solution:")
    x = np.zeros(n)
    for i, j in enumerate(basis):
        x[j] = xb[i]
    print(print_vector('x', x))
    print(print_basis(basis))
    return basis, B_inv, xb

In [16]:
def revised_simplex(A, b, c, basis, B_inv, xb):
    print("\n=== Revised Simplex Method ===")
    n = A.shape[1]
    iteration = 0
    
    while True:
        iteration += 1
        print(f"\nIteration {iteration}:")
        cb = c[basis]
        z = cb.dot(xb)
        x = np.zeros(n)
        for i, j in enumerate(basis):
            x[j] = xb[i]
        print(f"Corner Point: {print_vector('x', x)}")
        print_simplex_table(A, B_inv, basis, xb, c, z)
        
        # Compute reduced costs
        min_cj_bar = float('inf')
        entering = -1
        for j in range(n):
            if j not in basis:
                cj_bar = c[j] - cb.dot(B_inv.dot(A[:, j]))
                if cj_bar < min_cj_bar:
                    min_cj_bar = cj_bar
                    entering = j
        if min_cj_bar >= -1e-10:  # Optimality condition
            print("\nOptimal Solution Found:")
            print(f"Optimal Corner Point: {print_vector('x', x)}")
            print(f"Optimal Objective Value: z = {z:.4f}")
            print(print_basis(basis))
            return x, z, basis
        
        # Compute direction
        y = B_inv.dot(A[:, entering])
        if all(y <= 1e-10):
            print("Problem is unbounded")
            return None, None, None
        
        # Compute step length
        ratios = [xb[i] / y[i] if y[i] > 1e-10 else float('inf') for i in range(len(basis))]
        theta = min(r for r in ratios if r > 0)
        leaving_idx = ratios.index(theta)
        leaving = basis[leaving_idx]
        
        # Update basis
        basis[leaving_idx] = entering
        E = np.eye(len(basis))
        for i in range(len(basis)):
            if i != leaving_idx:
                E[i, leaving_idx] = -y[i] / y[leaving_idx]
            else:
                E[i, leaving_idx] = 1 / y[leaving_idx]
        B_inv = E.dot(B_inv)
        xb = B_inv.dot(b)

In [17]:
def solve_linear_program():
    # Problem data
    c = np.array([3, 1, 7, 6, 3], dtype=float)  # Objective function coefficients
    A = np.array([
        [-9, -3, 20, 23, -7],
        [-9, 15, -9, 23, 10]
    ], dtype=float)  # Constraint coefficients
    b = np.array([2, 8], dtype=float)  # Right-hand side
    m, n = A.shape
    
    # Step 1: Find initial basic feasible solution using artificial basis
    basis, B_inv, xb = artificial_basis(A, b, m, n)
    
    # Step 2: Optimize using revised simplex method
    x, z, basis = revised_simplex(A, b, c, basis, B_inv, xb)
    
    return x, z, basis

In [18]:
if __name__ == "__main__":
    x, z, basis = solve_linear_program()


=== Artificial Basis Method ===
Initial Basic Solution: x = [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]
Artificial Variables: x6, ..., x7
Basis: [5, 6]

Simplex Table:
Basic Variables: [5, 6]
Basic Solution: xb = [2.0000, 8.0000]
Objective Value: z = 10.0000
Reduced Costs (for non-basic variables):
c1_bar = 18.0000
c2_bar = -12.0000
c3_bar = -11.0000
c4_bar = -46.0000
c5_bar = -3.0000

Simplex Table:
Basic Variables: [3, 6]
Basic Solution: xb = [0.0870, 6.0000]
Objective Value: z = 6.0000
Reduced Costs (for non-basic variables):
c1_bar = 0.0000
c2_bar = -18.0000
c3_bar = 29.0000
c5_bar = -17.0000
c6_bar = 2.0000

Simplex Table:
Basic Variables: [3, 1]
Basic Solution: xb = [0.1304, 0.3333]
Objective Value: z = 0.0000
Reduced Costs (for non-basic variables):
c1_bar = 0.0000
c3_bar = 0.0000
c5_bar = 0.0000
c6_bar = 1.0000
c7_bar = 1.0000

Found Initial Basic Feasible Solution:
x = [0.0000, 0.3333, 0.0000, 0.1304, 0.0000]
Basis: [3, 1]

=== Revised Simplex Method ===

Iterati